In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from Dataloader import prepare_data_loader

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

In [ ]:
train_loader = prepare_data_loader(overlap=0.25, window_length=1, decimation_factor=3, spect_nfft=128, spect_nperseg=128, batch_size=32)

In [ ]:
train_loader.dataset.tensors[0].shape, train_loader.dataset.tensors[1].shape
height, width = train_loader.dataset.tensors[0].shape[1:]

In [ ]:
classifier = Classifier().to(device)
encoder = torch.load('./models/encoder.pth').to(device)

optimizer = torch.optim.Adam(classifier.parameters(), lr=1e-3, weight_decay=1e-5)
criterion = nn.MSELoss()

In [ ]:
num_epochs = 100
all_losses = []
classifier.train()
for epoch in range(num_epochs):
    for x, y in train_loader:
        x = x.to(device)
        y = y.to(device)
        embedding = encoder(x)
        embedding = embedding.view(embedding.size(0), -1)
        preds = classifier(embedding)
        loss = criterion(preds, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'epoch [{epoch + 1}/{num_epochs}], loss:{loss.item()}')

torch.save(classifier, './models/classifier.pth')
plt.plot(all_losses)
plt.show()

In [ ]:
from sklearn.model_selection import KFold

# Define the number of splits for cross-validation
num_folds = 5
kf = KFold(n_splits=num_folds, shuffle=True)

# Initialize lists to store results
fold_losses = []

# Cross-validation loop
for fold, (train_idx, val_idx) in enumerate(kf.split(x_data)):
    print(f'Fold {fold + 1}/{num_folds}')
    
    # Split data
    x_train, x_val = x_data[train_idx], x_data[val_idx]
    y_train, y_val = y_data[train_idx], y_data[val_idx]
    
    # Create data loaders
    train_loader = torch.utils.data.DataLoader(dataset=list(zip(x_train, y_train)), batch_size=128, shuffle=True)
    val_loader = torch.utils.data.DataLoader(dataset=list(zip(x_val, y_val)), batch_size=128, shuffle=False)
    
    # Initialize model, optimizer, and loss function
    classifier = Classifier().to(device)
    optimizer = torch.optim.Adam(classifier.parameters(), lr=1e-3, weight_decay=1e-5)
    criterion = nn.MSELoss()
    
    # Training loop
    classifier.train()
    for epoch in range(num_epochs):
        for x, y in train_loader:
            x = x.to(device)
            y = y.to(device)
            x = encoder(x)
            x = x.view(x.size(0), -1)
            preds = classifier(x)
            loss = criterion(preds, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    
    # Validation loop
    classifier.eval()
    val_loss = 0
    with torch.no_grad():
        for x, y in val_loader:
            x = x.to(device)
            y = y.to(device)
            x = encoder(x)
            x = x.view(x.size(0), -1)
            preds = classifier(x)
            loss = criterion(preds, y)
            val_loss += loss.item()
    
    val_loss /= len(val_loader)
    fold_losses.append(val_loss)
    print(f'Fold {fold + 1} validation loss: {val_loss}')

# Print average loss across all folds
print(f'Average validation loss: {np.mean(fold_losses)}')